## Clustering algorithms: nested matrices vs. inverted tables
Here we compare and contrast the two approaches outlined in the two clustering implementations using [nested matrices](nested.ipynb) and [inverted tables](inverted.ipynb).

## Memory use

### Representation
The size of each representation as follows:

|     Format     | Full data | Lengths subset |
| -------------- | --------- | -------------- |
| Nested matrix  |     278kB |           56kB |
| Inverted table |      74kB |           18kB |

In [3]:
]get ../csv/penguins_lter.csv
inverted←(1⌷penguins_lter)(↑¨↓⍉1↓penguins_lter)
⎕SIZE'penguins_lter'
⎕SIZE'inverted'
cols ← 'Culmen Length (mm)' 'Flipper Length (mm)' 'Species'
lengths←penguins_lter[;(1⌷penguins_lter)⍳⊆cols]
(head data)←inverted
I←{(⊂⍺)⌷⍵}
lengths_inv←(cols)((head⍳cols) I data)
⎕SIZE'lengths'
⎕SIZE'lengths_inv'

#.penguins_lter

278080

74056

55800

17960

### Computation
We will use the function `Memory` that estimates the memory required to run a function using the high water mark from [the memory manager](https://help.dyalog.com/latest/Content/Language/I%20Beam%20Functions/Memory%20Manager%20Statistics.htm).

In [10]:
∇ used←Memory expr
  {}⎕WA
  {}0(2000⌶)14
  hwm←2000⌶14
  {}⍎expr
  used←hwm-⍨2000⌶14
∇

## Compute Time
We will ignore the compute time of ingesting the data as usually this is a fixed cost and can probably be made comparable for the two formats. For convenience I am using the `]Get` user command which interprets CSV as a nested matrix by default, but we can easily imagine modifying this for inverted tables if needed.

In [71]:
∇ KMeans←{
⍝ ⍺: n←number of clusters :: scalar integer
⍝ ⍵: data set             :: numeric matrix 1 column per field
  n←⍺
  ComputeCentroids←{
    d←0.5*⍨+/2*⍨⍺(-⍤1⍤1 2)⍵   ⍝ distance from points to centroids
    g←d⍳⍤1 0⌊/d               ⍝ cluster (group) for each data point
    g{(+⌿÷1⌈≢)⍵}⌸⍺            ⍝ new clusters are means of points in each group
  }
  i←0
  I←{(⊂⍺)⌷⍵}   ⍝ ⊂⍛⌷      ⍝ Indexing
  c←n(?∘≢I⊢)⍵             ⍝ Guess random centroids
  ⍵ ComputeCentroids⍣≡c   ⍝ Compute centroids
}
∇

In [12]:
∇ KMeansInverted←{
⍝ ⍺: number of clusters :: scalar integer
⍝ ⍵: data set           :: inverted table
  n←⍺
  ComputeCentroids←{
    d←0.5*⍨⊃+/2*⍨⍺∘.-¨⍵        ⍝ distances from points to centroids
    g←d⍳⍤1 0⌊/d                ⍝ cluster (group) for each data point
    (⊂d⍳⍤1 0⌊/d){(+⌿÷≢)⍵}⌸¨⍺   ⍝ new clusters are means of points in each group
  }
  i←0
  I←{(⊂⊂⍺)⌷¨⍵}
  c←3(?∘≢∘⊃I⊢)⍵                ⍝ guess random centroids
  ⍵ ComputeCentroids⍣≡c   ⍝ Compute centroids
}
∇

⍳3

First we remove 0 values.

In [51]:
clean←(1 ¯1↓lengths)⌿⍨nz←∨/1≠(,0)⍳(1 ¯1↓lengths)
clean_inv←nz∘⌿¨2↑2⊃lengths_inv

In [52]:
3 KMeans clean

38.4275 186.9916667
45.95483871 196.7311828
47.56744186 216.8837209

In [57]:
]box on
3 KMeansInverted clean_inv

Was ON

┌───────────────────────────────┬───────────────────────────────────┐
│38.4275 45.95483871 47.56744186│186.9916667 196.7311828 216.8837209│
└───────────────────────────────┴───────────────────────────────────┘

In [58]:
]runtime -c "3 KMeans clean" "3 KMeansInverted clean_inv"

3 KMeans clean → 1.3E¯3 | 0% ⎕⎕⎕⎕⎕⎕⎕⎕⎕⎕⎕⎕⎕⎕⎕⎕⎕⎕⎕⎕⎕⎕⎕⎕⎕⎕⎕⎕⎕⎕⎕⎕⎕⎕⎕⎕⎕⎕⎕⎕ 
* 3 KMeansInverted clean_inv → 6.9E¯4 | -49% ⎕⎕⎕⎕⎕⎕⎕⎕⎕⎕⎕⎕⎕⎕⎕⎕⎕⎕⎕⎕⎕

## Ergonomics and Aesthetics
Dyalog out-of-the-box offers affordances to the nested matrix format in terms of notational convenience.

Indexing is leading-axis oriented, so selecting from a matrix feels natural. But since we write a cover for Select `{(⊂⍺)⌷⍵}`, a different cover for inverted tables isn't any worse.

The length of columns in an inverted table requires a disclose `≢∘⊃` vs simply `≢` for the matrix.

In both cases, the obvious comparison in mainstream languages is the Data Frame structure. In our case, we have opted to separate the header from the data in both formats. This must be done in the inverted tables, otherwise the benefit of storing homogeneous columns is lost. For nested matrices, we are simply trading `(1⌷table)⍳⊆column_names` for `head⍳⊆column_names`.

It's actually surprising how little of the soup has to change between the matrix and inverted table formats. Once the initial processing over records is done (in our case, outer product difference and use of key), you tend towards a simple intermediate array and usually a simple array result for statistical processes.

Additional encloses, discloses (firsts) and eaches are a little irksome. Take the "guess $n$ random centroids". There is a lovely ambivalent train.

```apl
(⊂⍤?∘≢⌷⊢)⍵    ⍝ Choose 1 from ⍵
n(⊂⍤?∘≢⌷⊢)⍵   ⍝ Choose n from ⍵
```

Modifications for an inverted table are a bit nasty

```apl
(⊂⍤⊂⍤?∘≢∘⊃⌷¨⊢)⍵         ⍝ Choose 1 from ⍵
n(⊂⍤⊂⍤?∘≢∘⊃⌷¨⊢)⍵        ⍝ Choose n from ⍵
n{⍺←1 ⋄ (⊂⍺?≢⍵)∘⌷¨⍵}⍵   ⍝ Maybe better as a dfn
```

The difference isn't as stark if we cover indexing.

```apl
I←{(⊂⍺)⌷⍵}
c←n(?∘≢I⊢)⍵
```

The equivalent for inverted tables.

```apl
I←{(⊂⍺)∘⌷¨⍵}
c←n(?∘≢∘⊃I⊢)⍵
```
